<h1 style='color:blue;'> Clasificación de texto </h1> 

Importar librerias de interés

In [1]:
import numpy as np
import pandas as pd

import re 
import string
import nltk


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

porter = PorterStemmer()
lancaster = LancasterStemmer()
wordnet_lemma = WordNetLemmatizer()

nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cx001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cx001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cx001\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


leer los archivos

In [2]:
train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('comentarios.csv',encoding='utf-16', sep=',',  error_bad_lines=False)

b'Skipping line 41116: expected 7 fields, saw 11\nSkipping line 41247: expected 7 fields, saw 11\nSkipping line 67200: expected 7 fields, saw 11\nSkipping line 69160: expected 7 fields, saw 11\n'


In [3]:
train.rename(columns={'identity_hate':'identity_attack'}, inplace=True)
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test.rename(columns={'comment':'comment_text'}, inplace=True)
test.head()

,id,comment_text,creation_time,source,tweet_id,user,user_id
0,0,RT @NoSurrenderHK: @guardiannews The truth is ...,2020-06-13 16:07:02,Twitter for Android,1271836487663808513,currentecalamo,1005852785609326592
1,1,RT @MarisaKabas: Content warning: police bruta...,2020-06-13 16:07:02,Twitter for Android,1271836487663775744,Donald Dire,1088300096666591232
2,2,RT @shahmiruk: It is absolutely unfair &amp; d...,2020-06-13 16:07:02,Twitter for iPhone,1271836487693275144,tanya cochrane 🕷#FBPE,25872176
3,3,RT @autotheoryqueen: TERFs police the boundari...,2020-06-13 16:07:02,Twitter for iPhone,1271836487747862529,Michael Bermingham,59031350
4,4,looks like a scary demon witch monster to me,2020-06-13 16:07:02,Twitter for iPhone,1271836487676579841,untossable chum,2771192143


Preparación de texto: se remueven signos de puntuación y se convierte todo el texto en minuscula

In [5]:
def clean_text_round1(text):
    '''make text lowercase, remove punctuation.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[\n]', ' ', text)
    text = re.sub('@\w+', ' ', text)
    text = re.sub('RT', ' ', text)
    return text

round1 = lambda x: clean_text_round1(str(x))

In [6]:
test.comment_text = test.comment_text.apply(round1)
train.comment_text = train.comment_text.apply(round1)

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [8]:
test.head()

,id,comment_text,creation_time,source,tweet_id,user,user_id
0,0,rt nosurrenderhk guardiannews the truth is hk ...,2020-06-13 16:07:02,Twitter for Android,1271836487663808513,currentecalamo,1005852785609326592
1,1,rt marisakabas content warning police brutalit...,2020-06-13 16:07:02,Twitter for Android,1271836487663775744,Donald Dire,1088300096666591232
2,2,rt shahmiruk it is absolutely unfair amp disin...,2020-06-13 16:07:02,Twitter for iPhone,1271836487693275144,tanya cochrane 🕷#FBPE,25872176
3,3,rt autotheoryqueen terfs police the boundaries...,2020-06-13 16:07:02,Twitter for iPhone,1271836487747862529,Michael Bermingham,59031350
4,4,looks like a scary demon witch monster to me,2020-06-13 16:07:02,Twitter for iPhone,1271836487676579841,untossable chum,2771192143


### Implementar modelo de clasificación

Este codigo se construyo usando la referencia: https://www.kaggle.com/thousandvoices/logistic-regression-with-words-and-char-n-grams

In [9]:
#Nombrar las clases
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_attack']

In [10]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

#### TF-IDF

In [11]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features=30000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)
vectorizer = make_union(word_vectorizer, char_vectorizer, n_jobs=2)

In [12]:
vectorizer.fit(all_text)
train_features = vectorizer.transform(train_text)
test_features = vectorizer.transform(test_text)

#### Generar Scores

In [13]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(
        classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9776134910411179
CV score for class severe_toxic is 0.9884755147227393
CV score for class obscene is 0.9901525490790254
CV score for class threat is 0.9881624549277713
CV score for class insult is 0.9824518061456994
CV score for class identity_attack is 0.9825912954858601
Total CV score is 0.984907851900369


In [14]:
submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0,0.028525,0.001610,0.008574,0.000802,0.009100,0.003459
1,1,0.006022,0.002773,0.013770,0.001169,0.016658,0.003206
2,2,0.581241,0.007748,0.068670,0.001182,0.119748,0.061484
3,3,0.024132,0.001585,0.008654,0.000426,0.009315,0.013586
4,4,0.274693,0.002532,0.029018,0.000760,0.050144,0.006064
...,...,...,...,...,...,...,...
88981,89995,0.086303,0.007142,0.037556,0.001303,0.021357,0.006865
88982,89996,0.049233,0.004548,0.014641,0.004317,0.030982,0.021515
88983,89997,0.079394,0.002554,0.010066,0.002549,0.017209,0.004658
88984,89998,0.022804,0.003485,0.009155,0.001104,0.010862,0.002725


In [15]:
submission = pd.merge(test, submission, on='id')
submission.head()

,id,comment_text,creation_time,source,tweet_id,user,user_id,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0,rt nosurrenderhk guardiannews the truth is hk ...,2020-06-13 16:07:02,Twitter for Android,1271836487663808513,currentecalamo,1005852785609326592,0.028525,0.001610,0.008574,0.000802,0.009100,0.003459
1,1,rt marisakabas content warning police brutalit...,2020-06-13 16:07:02,Twitter for Android,1271836487663775744,Donald Dire,1088300096666591232,0.006022,0.002773,0.013770,0.001169,0.016658,0.003206
2,2,rt shahmiruk it is absolutely unfair amp disin...,2020-06-13 16:07:02,Twitter for iPhone,1271836487693275144,tanya cochrane 🕷#FBPE,25872176,0.581241,0.007748,0.068670,0.001182,0.119748,0.061484
3,3,rt autotheoryqueen terfs police the boundaries...,2020-06-13 16:07:02,Twitter for iPhone,1271836487747862529,Michael Bermingham,59031350,0.024132,0.001585,0.008654,0.000426,0.009315,0.013586
4,4,looks like a scary demon witch monster to me,2020-06-13 16:07:02,Twitter for iPhone,1271836487676579841,untossable chum,2771192143,0.274693,0.002532,0.029018,0.000760,0.050144,0.006064


Cargar los datos de salida a un CSV

In [16]:
submission.to_csv('submission.csv', index=False)